# This notebook describes the analysis of the babies_mg dataset

In [ ]:
# Convert biom file to genus file

# Summary table is from Nan's folder: /sc/arion/projects/clemej05a/cooccurrence/longi_test_data/biom/maturation/taxa/, copied to /sc/arion/projects/clemej05a/hilary/babies_mg/taxa/

# On Minerva
source activate qiime2-2020.8.0

biom convert -i otu_table_merged.0-12m.stool.child.biom -o otu_table_merged.0-12m.stool.child.tsv

# Summarize taxa results in /taxa/ folder. Using L6 for downstream analyses

In [ ]:
# Convert genus level table to trajectories

python sc_create_trajs_babies.py

In [ ]:
# Cluster UNSMOOTHED trajectories
# NOTE: These scripts are located in the analysis_scripts folder in loclust

# Make cluster commands
python sc_create_j_clust.py -i "/sc/arion/projects/clemej05a/hilary/babies_mg/otu_tables_and_mapping_files/taxa_trajs_stool_child_months_lego_interp/" -b "/sc/arion/projects/clemej05a/hilary/babies_mg_lego5_outputs/" -s "/sc/arion/projects/clemej05a/hilary/repos/loclust/scripts/" -o Babies_lego5_commands_for_jobs.txt 

# Make and submit jobs
python generate_lsf_loclust.py -e loclust3pt8 -c Babies_lego5_commands_for_jobs.txt -o /sc/arion/projects/clemej05a/hilary/babies_mg_lego5_outputs/jobs/ -py /hpc/users/monach01/.conda/envs/loclust3pt8/lib/python3.8/ --submit

In [ ]:
# Smooth trajectories

python sc_interp_babies_trajs_lego.py

In [ ]:
# Cluster trajectories
# NOTE: These scripts are located in the analysis_scripts folder in loclust

# Make cluster commands
python sc_create_j_clust.py -i "/sc/arion/projects/clemej05a/hilary/babies_mg/taxa_trajs_lego_interp/" -b "/sc/arion/projects/clemej05a/hilary/babies_mg/" -s "/sc/arion/projects/clemej05a/hilary/repos/loclust/scripts" -o Babies_lego5_commands_for_jobs.txt

# Make and submit jobs
python generate_lsf_loclust.py -e loclust3pt8 -c Babies_lego5_commands_for_jobs.txt -o /sc/arion/projects/clemej05a/hilary/babies_mg/jobs_lego5/ -py /hpc/users/monach01/.conda/envs/loclust3pt8/lib/python3.8/ --submit

In [ ]:
# Calculate silhouette scores
# NOTE: These scripts are located in the analysis_scripts folder in loclust
python sc_stdout_parse.py -i '/sc/arion/projects/clemej05a/hilary/babies_mg/jobs/'
python sc_stdout_parse.py -i '/sc/arion/projects/clemej05a/hilary/babies_mg/jobs_lego5/'

# Calculate v-measure
python sc_vmeasure_with_mdata.py -i '/sc/arion/projects/clemej05a/hilary/babies_mg/outputs/taxa_trajs/' -m "delivery"
python sc_vmeasure_with_mdata.py -i '/sc/arion/projects/clemej05a/hilary/babies_mg/outputs/taxa_trajs_lego_interp/' -m "delivery"

# Calculate chi squared results
python sc_chiSq.py -i '/sc/arion/projects/clemej05a/hilary/babies_mg/outputs/taxa_trajs/' -m "delivery"
python sc_chiSq.py -i '/sc/arion/projects/clemej05a/hilary/babies_mg/outputs/taxa_trajs_lego_interp/' -m "delivery"


Review sc_stdout_parse.py outputs for selection of clusters by silhouette score. 

Corresponding info the clusters of interest will be in the output files form sc_vmeasure_with_mdata.py and sc_chiSq.py

Transfer relevant cluster files as appropriate before plotting. 

In [ ]:
# Plotting

# Convert traj files to tidy data format for plotting in R
# NOTE: This script is located in the analysis_scripts folder in loclust
python sc_convert_traj_for_R_plotting.py -i 'C:/Users/mathr/Documents/GitHub/babies_mg/otu_tables_and_mapping_files/taxa_trajs_lego_interp/' -m "delivery" -o 'C:/Users/mathr/Documents/GitHub/babies_mg/otu_tables_and_mapping_files/taxa_trajs_lego_interp/_R/'

# Plot trajs and associated clusters colored by delivery metadata in R
R sc_plot_babies_trajs.R

# Combine v-meausre and silhoeutte score information into a single dataframe
python sc_vmeasure_with_mdata_merge.py

# Plot V-measure vs silhouette score
R sc_babies_only_silhouette_vs_vmeasure.R
